# Weaviate v1.26 code samples - Multi-target vector search

This notebook contains sample code to setup a sample collection with asynchronous repair enabled. 

Additional information:
- [Weaviate documentation](https://weaviate.io/developers/weaviate)
- [Weaviate v1.26 release notes](https://weaviate.io/developers/weaviate/release-notes/release_1_26)

In [12]:
# CREATE A CLIENT (LOCALHOST)

import os
import weaviate

client = weaviate.connect_to_local(
    headers={"X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"],}
)

# Uncomment to check the connection
# client.is_ready()

MULTI-TARGET VECTORS SETUP

- Define a collection that has multiple vectors 
- Upload data to the collection

In [32]:
# DEFINE CONSTANTS
collection = "JeopardyTiny"

In [ ]:
# DEFINE THE COLLECTION

from weaviate.classes.config import Configure, Property, DataType

# Remove old version of this collection if there is one
if (client.collections.exists(collection)):
    client.collections.delete(collection)
    # print(f"Removed old collection: {collection}")

# Define a new schema
client.collections.create(
    name=collection,
    description="Jeopardy game show questions",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="jeopardy_questions_vector",
            source_properties=["question"],
            vectorize_collection_name=False,
        ),
        Configure.NamedVectors.text2vec_openai(
            name="jeopardy_answers_vector",
            source_properties=["answer"],
            vectorize_collection_name=False,
        ),
    ],
    properties=[
        Property(name="category", data_type=DataType.TEXT),
        Property(name="question", data_type=DataType.TEXT),
        Property(name="answer", data_type=DataType.TEXT),
    ],
)

## CHECK VALUES - uncomment the next line to see the collection definition
# print(client.collections.get(collection))

In [21]:
# UPLOAD THE DATA

# Uncomment to install the requests module
# !pip install requests
import json, requests


# Get the sample data set
resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json"
)
data = json.loads(resp.text)

# Prepare the sample data for upload
question_objects = list()
for row in data:
    question_objects.append(
        {
            "question": row["Question"],
            "answer": row["Answer"],
            "category": row["Category"],
        }
    )

# Upload the sample data
jeopardy = client.collections.get(collection)
with jeopardy.batch.dynamic() as batch:
    for q in question_objects:
        batch.add_object(properties=q)


In [ ]:
# PRINT THE FIRST FEW OBJECTS
import pprint as pp

response = jeopardy.query.fetch_objects(
        include_vector=True,
        limit=1
        )

# Uncomment to see the first object
# for o in response.objects:
#     pp.pprint(o.properties)
#     for k,v in o.vector.items():
#         print(f"vector: {k}  {v}")

MULTI-TARGET VECTOR QUERY EXAMPLES

In [ ]:
# BASIC QUERY

jeopardy = client.collections.get(collection)
response = jeopardy.query.near_text(
    query="a wild animal",
    limit=2,
    target_vector=["jeopardy_questions_vector", "jeopardy_answers_vector"],  # Specify the target vectors
)

for o in response.objects:
    print(o.properties)


In [ ]:
### MULTI-TARGET QUERY, ADJUST RELATIVE WEIGHTS

from weaviate.classes.query import TargetVectors

jeopardy = client.collections.get(collection)
response = jeopardy.query.near_text(
    query="a wild animal",
    limit=2,
    target_vector=TargetVectors.manual_weights({"jeopardy_questions_vector": 5, "jeopardy_answers_vector": 0.3})
)

for o in response.objects:
    print(o.properties)
